In [ ]:
import os
import pandas as pd
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 15)

import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12, 'font.family': 'Candara'})
plt.rcParams["figure.dpi"] = 130
plt.rcParams["figure.figsize"] = (7, 4.5)
plt.style.use('ggplot')
from filter import DSPreprocess, plot_panel_data

data_path      = r"D:\Datastream\US"
save_dir = "Filtered_Data"
os.makedirs(data_path, save_dir)
nof_subfolders = 31 

timeseries_dfs = []
static_dfs     = []

for i in tqdm(range(1, nof_subfolders + 1), desc="Load data"):
    folder_nbr       = f"{i:02d}"
    folder_path      = os.path.join(data_path, folder_nbr)
    OHLCV_panel_iter = pd.read_feather(os.path.join(data_path, "Panel_Data_US", f"OHLCV_panel_{folder_nbr}.feather"))
    static_iter      = pd.read_excel(os.path.join(folder_path, f'static_{folder_nbr}.xlsx'), engine='openpyxl')
    timeseries_dfs.append(OHLCV_panel_iter)
    static_dfs.append(static_iter)

statics = pd.concat(static_dfs, axis=0, ignore_index=True)
statics.reset_index(drop=True, inplace=True)

delist_str = statics["ENAME"].str.extract(r"DELIST\.(\d{2}/\d{2}/\d{2})")[0]
statics["Delisting Date"] = pd.to_datetime(delist_str, format="%d/%m/%y", errors="coerce")

statics['BDATE'] = pd.to_datetime(statics['BDATE'])
string_columns = ['DSCD', 'ENAME', 'EXMNEM', 'GEOGN', 'ISIN', 'ISINID', 'LOC', 'PCUR', 'TRAC', 'TYPE', 'CURRENCY']
if 'Type' in statics.columns:
    string_columns.insert(0, 'Type')

statics[string_columns] = statics[string_columns].astype(str)

OHLCV_panel = pd.concat(timeseries_dfs, axis=0, ignore_index=True)
OHLCV_panel.sort_values(by=["Date", "Stock"], inplace=True)
OHLCV_panel.reset_index(drop=True, inplace=True)

print(f"Number of rows before removing duplicate Stock-Date observations: {OHLCV_panel.shape[0]}")
OHLCV_panel = OHLCV_panel.drop_duplicates(subset=["Stock", "Date"], keep="first")
print(f"Number of rows after removing duplicate Stock-Date observations: {OHLCV_panel.shape[0]}")

OHLCV_panel.loc[:, 'Stock'] = OHLCV_panel['Stock'].astype(str)
OHLCV_panel.loc[:, 'Stock'] = OHLCV_panel['Stock'].str.strip()
statics.loc[:, 'DSCD']      = statics['DSCD'].str.strip()

mask = OHLCV_panel["ReturnIndex"] < 1e-6
print(f"Frequency of ReturnIndex observations with extreme small values: {mask.sum()/OHLCV_panel.shape[0]:.6f}")

OHLCV_panel.loc[mask, "ReturnIndex"] = np.nan

OHLCV_panel["Return"] = (
    OHLCV_panel.groupby("Stock")["ReturnIndex"]
    .transform(lambda x: x / x.shift(1) - 1)
)

print(f"Number of companies before removing non-regional companies: {statics.shape[0]}")
statics = statics[statics['GEOGN'] == 'UNITED STATES']
print(f"Number of companies after removing non-regional companies: {statics.shape[0]}")

########################################################################################################################
## Filters based on static data
########################################################################################################################
# Filter (1) - Equity filter:
OHLCV_panel = DSPreprocess.filter_non_common_stocks(OHLCV_panel, statics, country='UNITED STATES')


# Filter (2) - Cross-listing filter:
OHLCV_panel = DSPreprocess.filter_cross_listings(OHLCV_panel, statics, country='UNITED STATES')


# Filter (3): Duplicate LOC Codes
OHLCV_panel = DSPreprocess.filter_duplicate_loc_codes(OHLCV_panel, statics)


# Filter (4) - Foreign firms:
OHLCV_panel = OHLCV_panel[OHLCV_panel.Stock.isin(statics.DSCD.unique())]


# Filter (5) - Stocks in foreign currencies:
OHLCV_panel = DSPreprocess.filter_foreign_currency_stocks(OHLCV_panel, statics, country='UNITED STATES')


# Filter (17) - Survivorship bias (obsolete for US data, because our dataset starts in 1993):
# OHLCV_panel = DSPreprocess.filter_surivorship_bias(OHLCV_panel, statics, country='UNITED STATES')


########################################################################################################################
## Filters based on ReturnIndex
########################################################################################################################
# Filter (7) - :
# Remove stocks of which more than 98% of non-zero mean returns are either positive or negative
OHLCV_panel = DSPreprocess.filter_implausible_returns(OHLCV_panel)

########################################################################################################################
## Stockday filters:
########################################################################################################################
# Filter (13):
# If RI is forward filled for 10 consecutive days, then remove those days.
OHLCV_panel = DSPreprocess.filter_padded_values_delistings(OHLCV_panel, statics)


# Filter (8):
# Remove stocks for which the returns are zero in more than 95% of their sample (After applying filter (13).
OHLCV_panel = DSPreprocess.filter_zero_return_stocks(OHLCV_panel)


# Filter (14):
# Stale prices
OHLCV_panel = DSPreprocess.filter_stale_prices(OHLCV_panel)


# Filter (9):
# Remove stocks with a daily standard deviation of more than 40%.
OHLCV_panel  = DSPreprocess.filter_stocks_by_high_volatility(OHLCV_panel, volatility_threshold=0.40)


# Filter (10):
# Remove stocks with a daily standard deviation of less than 0.01 bps.
OHLCV_panel = DSPreprocess.filter_stocks_by_low_volatility(OHLCV_panel)


# Filter (15):
# Target filter rate not reported / ~0.0015% (~0.00569% when applied on raw panel) actual filter rate
OHLCV_panel = DSPreprocess.filter_outlier_errors(OHLCV_panel, up_ts=1.0, down_ts=-0.5, method='drop')


# Filter (16):
# Holiday filter: Has to be applied after filter (11) and (13)!
# Remove days on which non-missing or non-zero returns account for less than 0.5% of total available stocks.
OHLCV_panel = DSPreprocess.filter_holidays(OHLCV_panel)


# Filter (Own - implausible OHLC):
# Nonsense values (Low > (Open OR High OR Close) and High < (Open OR Low OR Close):
OHLCV_panel = DSPreprocess.filter_implausible_prices(OHLCV_panel)


# Filter (Extreme prices - Schmidt, von Arx (2011))
# Remove prices higher than 1mio US$.
# OHLCV_panel = DSPreprocess.filter_extreme_prices(OHLCV_panel, ts=1_000_000)


# NOT NEEDED - Filter (20 - Extreme returns due to decimal errors - Annaert et al. (2013) JBF)
# OHLCV_panel = DSPreprocess.filter_decimal_errors(OHLCV_panel, up_ts=4.0, down_ts=-0.85)


# Filter (No trading activity - Chaieb et al. (2021) JoFE)
OHLCV_panel = DSPreprocess.filter_no_trading_activity(OHLCV_panel)

#
# # Filter (Own - Extreme returns)
# # OHLCV_panel = DSPreprocess.filter_extreme_returns(OHLCV_panel, lower=0.00, upper=0.999)
# OHLCV_panel = DSPreprocess.filter_extreme_returns2(OHLCV_panel, n_std=5) # Less aggressive than above version.


# Filter (Own - NA filter) - Drop all rows before they are populated for the first time and apply forward + backward fill.
OHLCV_panel = DSPreprocess.handle_missings(OHLCV_panel, statics, country='UNITED STATES')


########################################################################################################################
# Manual removal of implausibilities:
########################################################################################################################
# 1.) AgEagle Aerial Systems, Inc. (680683). Remove observations before foundation date.
OHLCV_panel = OHLCV_panel[~((OHLCV_panel['Stock'] == "680683") & (OHLCV_panel['Date'] < '2010-01-01'))]


# 2.) Strange prices due to stock splits / reverse splits
OHLCV_panel = OHLCV_panel[~(OHLCV_panel["Stock"]  == "872328")]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "9364PF") & (OHLCV_panel["Date"] == "2020-07-13"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "50259R") & (OHLCV_panel["Date"] == "2018-01-04"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "67684T") & (OHLCV_panel["Date"] == "2023-08-07"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "2566DU") & (OHLCV_panel["Date"] == "2024-06-04"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "28355P") & (OHLCV_panel["Date"] == "2008-12-17"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "2634G3") & (OHLCV_panel["Date"] == "2024-03-27"))]
OHLCV_panel = OHLCV_panel[~((OHLCV_panel["Stock"] == "32650J") & (OHLCV_panel["Date"] == "2009-05-07"))]


# 3.) Implausible returns due to high amount of missings:
OHLCV_panel = OHLCV_panel[~(OHLCV_panel["Stock"] == "7076TJ")]
OHLCV_panel = OHLCV_panel[~(OHLCV_panel["Stock"] == "7076TK")]


# 4.) Seems to be some sort of dividend split, that was incorrectly labeled as a stock:
OHLCV_panel = OHLCV_panel[~(OHLCV_panel["Stock"] == "92238K")]


# 5.) Delete day with unusual drop in listed firms:
# OHLCV_panel = OHLCV_panel[~(OHLCV_panel["Date"] == '1995-05-26')]

# Filter (18) - Adjustment inconsistencies.
OHLCV_panel_temp = DSPreprocess.filter_adjustment_inconsistencies(OHLCV_panel, threshold=0.05)


# Filter (21) - Penny stocks.
OHLCV_panel = DSPreprocess.filter_penny_stocks(OHLCV_panel)


# Filter (Own - Extreme returns)
# OHLCV_panel = DSPreprocess.filter_extreme_returns(OHLCV_panel, lower=0.00, upper=0.999)
OHLCV_panel = DSPreprocess.filter_extreme_returns2(OHLCV_panel, n_std=7.5) # Less aggressive than above version.


# Filter (12) - Filters the panel to include only stocks with sufficient observation history
OHLCV_panel = DSPreprocess.filter_short_history_stocks(OHLCV_panel, threshold=120)


OHLCV_panel.replace([np.inf, -np.inf], np.nan, inplace=True)
OHLCV_panel_final = OHLCV_panel.dropna(subset=['Return'])

# extract companies which are in the final filtered data set
statics_for_filtered = statics[statics.DSCD.isin(OHLCV_panel_final.Stock.unique().tolist())]

# save
OHLCV_panel_final.to_feather(os.path.join(data_path, save_dir, "Financial_base_data_panel_filtered.feather"))
statics_for_filtered.to_csv(os.path.join(data_path, save_dir, "statics_filtered.csv"), index = False)

Load data: 100%|██████████| 31/31 [00:26<00:00,  1.17it/s]


Number of rows before removing duplicate Stock-Date observations: 142129385
Number of rows after removing duplicate Stock-Date observations: 142129385
Frequency of ReturnIndex observations with extreme small values: 0.000096
Number of companies before removing non-regional companies: 26542
Number of companies after removing non-regional companies: 26542
For UNITED STATES, filter (1) removes ~10.0% of stocks (based on raw data).
For UNITED STATES, filter (2) removes ~0.4% of stocks (based on raw data).
Filter (3) removes ~0.4% of stocks (based on raw data).
For UNITED STATES, filter (5) removes ~0.0% of stocks
Filter (7) removes ~0.1% of observations
Filter (13) removes ~9.9% of observations
Filter (8) removes ~1.0376% of observations
Filter (14) removes ~29.868% of observations
Filter (9) removes ~0.8999999999999999% of observations
Filter (10) removes ~0.061263% of observations
Filter (15) removes ~0.00161% of observations
Filter (16) removes ~3.461% of observations
OHLC inconsistency

In [30]:
statics_for_filtered.to_csv(os.path.join(data_path, save_dir, "statics_filtered.csv"), index = False)

In [32]:
OHLCV_panel_final.columns

Index(['Date', 'Stock', 'Open', 'High', 'Low', 'Close', 'Volume',
       'ReturnIndex', 'MCAP', 'MTBV', 'AdjFactor', 'UnadjClose', 'Return',
       'DSCD', 'Delisting Date', 'prev_UnadjClose'],
      dtype='object')

In [28]:
os.makedirs(os.path.join(data_path, save_dir))

FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'D:\\Datastream\\US\\Filtered_Data'

,Type,DSCD,BDATE,ENAME,EXMNEM,GEOGN,ISIN,ISINID,LOC,PCUR,TRAC,TYPE,WC05601,CURRENCY,Delisting Date
0,69568X,69568X,2010-06-29,TESLA,NAS,UNITED STATES,US88160R1014,P,U88160R101,U$,ORD,EQ,TSLA,U$,NaT
1,906150,906150,1973-01-02,GE AEROSPACE,NYS,UNITED STATES,US3696043013,P,U369604301,U$,ORD,EQ,GE,U$,NaT
2,905122,905122,1973-01-02,RTX,NYS,UNITED STATES,US75513E1010,P,U75513E101,U$,ORD,EQ,RTX,U$,NaT
3,904818,904818,1973-01-02,BOEING,NYS,UNITED STATES,US0970231058,P,U097023105,U$,ORD,EQ,BA,U$,NaT
4,154423,154423,1995-03-16,LOCKHEED MARTIN,NYS,UNITED STATES,US5398301094,P,U539830109,U$,ORD,EQ,LMT,U$,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26474,8702Y3,8702Y3,2015-11-10,PACE HOLDINGS DEAD - DELIST.13/03/17,NAS,UNITED STATES,KYG6865N1034,P,UG6865N103,U$,ORD,EQ,PACEU,U$,2017-03-13
26480,544885,544885,1991-05-20,PRIZE ENERGY RESOURCES DEAD - MERGER 357328,ASE,UNITED STATES,US74267L1061,P,U74267L106,U$,nan,EQ,NaN,U$,NaT
26520,326043,326043,1992-06-17,TAVA TECHNOLOGIES DEAD - MERGER 885888,NAS,UNITED STATES,US8721691074,P,U872169107,U$,nan,EQ,NaN,U$,NaT
26534,9067PR,9067PR,2017-04-28,VANTAGE ENERGY ACQUISITION CLASS A DEAD - DELI...,NAS,UNITED STATES,US92211L1052,P,U92211L105,U$,ORD,EQ,VEACU,U$,2019-04-16


In [20]:
desc = OHLCV_panel_final[['Open', 'High', 'Low', 'Close', 'Volume','ReturnIndex', 'MCAP', 'Return',]].describe(percentiles=[0.001, 0.01, 0.05, 0.95, 0.99, 0.999])


In [21]:
desc

,Open,High,Low,Close,Volume,ReturnIndex,MCAP,Return
count,2.750410e+07,2.750410e+07,2.750410e+07,2.750410e+07,2.750410e+07,2.750410e+07,2.750410e+07,2.750410e+07
mean,1.085794e+07,1.127789e+07,1.021349e+07,1.068042e+07,1.415309e+03,6.543992e+03,5.859633e+03,5.143119e-04
std,8.349681e+09,8.698349e+09,7.785818e+09,8.162660e+09,1.536250e+04,1.224350e+05,3.887156e+04,2.792502e-02
min,1.000000e-02,3.554700e-02,1.300000e-03,3.411500e-02,0.000000e+00,2.000000e-06,1.895200e-02,-6.194173e-01
0.1%,7.087500e-01,7.277020e-01,6.944730e-01,7.109380e-01,0.000000e+00,5.935420e-01,6.291598e+00,-1.566816e-01
1%,2.568078e+00,2.624999e+00,2.516734e+00,2.569998e+00,9.300000e-05,1.247875e+01,1.844806e+01,-7.865134e-02
5%,5.969533e+00,6.082549e+00,5.833333e+00,5.969999e+00,5.999960e-01,5.562013e+01,4.487189e+01,-3.983746e-02
50%,2.056000e+01,2.089542e+01,2.025000e+01,2.056250e+01,1.539964e+02,3.003225e+02,6.517241e+02,0.000000e+00
95%,1.183700e+02,1.200000e+02,1.166600e+02,1.183437e+02,4.574172e+03,1.443802e+04,2.060390e+04,4.177776e-02
99%,4.374195e+02,4.443997e+02,4.299834e+02,4.371299e+02,1.664428e+04,7.252575e+04,9.632262e+04,8.603077e-02
